# Data Processing

In [314]:
using CSV, DataFrames, DecisionTree, Statistics, Dates, Gadfly, Random, MLBase;
include("utils/precipitation.jl");

## Build features

### Get and filter the features

#### Latitude, Longitude, Height

In [317]:
features = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_Env", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(features, Symbol.(colnames));
select!(features, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]);

#### Replace missing Z index with mean

In [318]:
features.TP_Z = coalesce.(features.TP_Z, mean(features[completecases(features), :].TP_Z));
first(shuffleDf(features), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z
,String,Float64,Float64,Float64
1,3580-01D,45.4085,-73.9569,25.48
2,4796-01D,45.45,-73.5686,15.747
3,4230-07D,45.6888,-73.4863,9.86
4,3305-03D,45.6196,-73.6329,11.76
5,3750-01D,45.6087,-73.6397,11.6
6,4320-01D,45.5488,-73.5307,13.68
7,4315-01D,45.5628,-73.5243,14.39
8,0801-05D,45.5173,-73.5312,12.7
9,3480-03D,45.4972,-73.7953,25.51


### Load dates and surverses

In [321]:
surverses = CSV.read("data/surverses.csv",missingstring="-99999");

#### Filter months

In [322]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

#### Filter non rain surverses

In [323]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

#### Remove missing data and rename

In [324]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
first(shuffleDf(surverses),10)

,ID_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,4230-03D,2014-08-30,0
2,4370-02D,2014-06-01,0
3,3350-08D,2018-06-30,0
4,4350-01D,2014-06-06,0
5,4340-03D,2017-05-02,0
6,3450-02D,2014-08-13,0
7,4320-01D,2014-10-30,0
8,3530-01D,2014-08-09,0
9,0801-01D,2015-08-15,0


### Augment features with dates and label

In [325]:
comb = join(features, surverses, on = :ID_OUVRAGE);
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,SURVERSE
,String,Float64,Float64,Float64,Date,Int64
1,4610-08D,45.4267,-73.8538,19.3526,2017-09-24,0
2,0801-03D,45.5081,-73.5273,11.94,2017-07-17,0
3,4430-03D,45.4149,-73.6303,22.25,2015-09-25,0
4,4230-01D,45.6621,-73.4983,11.79,2018-07-16,0
5,3310-01D,45.6139,-73.6313,19.58,2015-10-06,0
6,4400-01D,45.4586,-73.5621,16.8,2014-06-12,1
7,4560-01D,45.4509,-73.7843,26.37,2013-06-06,0
8,3350-02D,45.5783,-73.6585,19.3526,2015-05-07,0
9,4530-02D,45.4408,-73.7711,19.3526,2013-07-26,0


### Load precipitation data

#### Load and filter months between May & October included

In [326]:
precipitation = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitation, Symbol("St-Hubert")=>:StHubert);

precipitation = filter(row -> month(row.date) > 4, precipitation);
precipitation = filter(row -> month(row.date) < 11, precipitation); 

#### Replace missing data by 0

In [327]:
precipitation[!,:McTavish] = coalesce.(precipitation[:,:McTavish], 0);
precipitation[!,:Bellevue] = coalesce.(precipitation[:,:Bellevue], 0);
precipitation[!,:Assomption] = coalesce.(precipitation[:,:Assomption], 0);
precipitation[!,:Trudeau] = coalesce.(precipitation[:,:Trudeau], 0);
precipitation[!,:StHubert] = coalesce.(precipitation[:,:StHubert], 0);

first(shuffleDf(precipitation), 5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2013-06-01,19,0,0,0,0,0
2,2013-08-14,2,36,3,0,16,0
3,2014-07-27,17,0,12,0,0,0
4,2019-10-08,17,0,0,0,0,0
5,2015-09-20,23,0,0,0,0,0


### Extract features from precipitation

#### Sum of precipitation for the day

In [328]:
pcp_sum = by(precipitation, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum);
first(shuffleDf(pcp_sum), 5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2014-07-17,0,7,0,0,0
2,2016-07-03,2,0,20,13,0
3,2015-08-23,0,0,0,0,0
4,2019-08-23,0,0,0,0,0
5,2015-08-30,0,0,0,0,0


#### Maximum precipitation in an hour for the day

In [329]:
pcp_max = by(precipitation, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(shuffleDf(pcp_max),5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2018-08-20,2,0,4,0,0
2,2018-07-06,32,0,31,0,25
3,2016-06-28,106,44,40,20,20
4,2018-08-27,0,0,6,2,2
5,2014-07-17,0,7,0,0,0


#### Maximum precipitation during three consecutive hours in a day

In [330]:
pcp_max3h = by(precipitation, :date,  McTavish = :McTavish=>maximum3, Bellevue = :Bellevue=>maximum3, 
   Assomption = :Assomption=>maximum3, Trudeau = :Trudeau=>maximum3, StHubert = :StHubert=>maximum3)
first(shuffleDf(pcp_max3h),5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-10-07,154,84,110,113,0
2,2016-08-29,0,0,0,0,0
3,2013-10-26,24,17,20,12,0
4,2015-07-25,0,0,10,0,0
5,2015-05-01,0,0,0,0,0


### Add precipitation data to features

#### Get stations lat-lng

In [331]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish", 45.504742, -73.579167]);
push!(station_df, ["Bellevue", 45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau", 45.467778, -73.741667]);
push!(station_df, ["StHubert", 45.5175, -73.416944]);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,45.5047,-73.5792
2,Bellevue,45.4272,-73.9292
3,Assomption,45.8094,-73.4347
4,Trudeau,45.4678,-73.7417
5,StHubert,45.5175,-73.4169


### Normalize elements

#### Get extreme values

In [332]:
min_lat = min(minimum(comb.TP_LAT), minimum(station_df.LAT));
max_lat = max(maximum(comb.TP_LAT), maximum(station_df.LAT));

min_lng = min(minimum(comb.TP_LNG), minimum(station_df.LNG));
max_lng = max(maximum(comb.TP_LNG), maximum(station_df.LNG));

min_z = minimum(comb.TP_Z);
max_z = maximum(comb.TP_Z);

#### Normalize every value of comb between 0 and 1

In [333]:
comb.TP_LAT = normalize.(comb.TP_LAT, min_lat, max_lat);
comb.TP_LNG = normalize.(comb.TP_LNG, min_lng, max_lng);
comb.TP_Z = normalize.(comb.TP_Z, min_z, max_z);

first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,SURVERSE
,String,Float64,Float64,Float64,Date,Int64
1,3540-01D,0.18804,0.152837,0.359859,2016-10-07,0
2,4230-06D,0.687093,0.864544,0.0203708,2016-09-21,0
3,4230-04D,0.660589,0.861562,0.0200438,2018-08-01,0
4,4430-01D,0.0736379,0.682312,0.360102,2018-09-07,0
5,3410-02D,0.319462,0.483178,0.359859,2013-09-28,0
6,3240-02D,0.64351,0.734083,0.0576464,2018-08-10,0
7,0801-01D,0.286346,0.795322,0.175359,2014-06-05,0
8,4560-03D,0.121422,0.322806,0.589314,2017-06-07,0
9,4620-06D,0.0789395,0.164927,0.774384,2014-06-12,0


#### Normalize every value of station_df between 0 and 1

In [334]:
station_df.LAT = normalize.(station_df.LAT, min_lat, max_lat);
station_df.LNG = normalize.(station_df.LNG, min_lng, max_lng);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,0.251316,0.699536
2,Bellevue,0.0608415,0.0512791
3,Assomption,1.0,0.967072
4,Trudeau,0.160492,0.39856
5,StHubert,0.282664,1.0


### Augment Features

#### Add pcp_sum and pcp_max columns

In [348]:
comb.PCP_SUM = zeros(size(comb, 1));
comb.PCP_MAX = zeros(size(comb, 1));
comb.PCP_MAX3 = zeros(size(comb, 1));
permutecols!(comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE]);

In [349]:
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,3260-01D,0.609996,0.697487,0.386587,2015-06-09,0.0,0.0,0.0
2,4230-06D,0.687093,0.864544,0.0203708,2015-07-15,0.0,0.0,0.0
3,0672-01D,0.687609,0.788765,0.0252755,2013-06-15,0.0,0.0,0.0
4,3370-01D,0.400013,0.544048,0.322499,2013-09-16,0.0,0.0,0.0
5,3350-11D,0.320282,0.463777,0.553674,2015-07-09,0.0,0.0,0.0
6,0801-04D,0.246824,0.802071,0.131217,2018-09-06,0.0,0.0,0.0
7,4265-01D,0.549187,0.847845,0.112252,2018-05-01,0.0,0.0,0.0
8,3580-01D,0.0148256,0.0,0.560213,2013-05-08,0.0,0.0,0.0
9,3275-02D,0.5763,0.647929,0.202171,2017-06-09,0.0,0.0,0.0


#### Find closest station to each ouvrage and add pcp_sum and pcp_max to it

In [350]:
for i=1:size(comb, 1)
    id_ouvrage = comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(comb[i, :TP_LAT], comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist); 
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
    # Augment comb with a weighted p_max3h, based on the distance to the station
    p_max3 = pcp_max3h[∈([comb[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
    comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
end

### TODO: Remove outlier in PCP_SUM and PCP_MAX AND PCP_MAX3 that cause compression

In [351]:
comb[comb[:PCP_SUM] .> 750, :PCP_SUM] = 750;
comb[comb[:PCP_MAX] .> 500, :PCP_MAX] = 500;
comb[comb[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[351]:1
└ @ Core In[351]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[351]:2
└ @ Core In[351]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[351]:3
└ @ Core In[351]:3


In [352]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4340-01D,0.304064,0.76384,0.359121,2017-08-04,135.691,91.683,102.685
2,4430-05D,0.0529114,0.554875,0.729588,2015-06-12,85.8857,24.3073,42.9428
3,4320-01D,0.359606,0.789317,0.174378,2016-10-21,415.917,54.138,128.9
4,3310-01D,0.51946,0.602926,0.367296,2016-08-13,187.325,63.6335,82.223
5,3350-11D,0.320282,0.463777,0.553674,2016-06-28,64.5382,16.5483,30.6143
6,4230-02D,0.655557,0.861448,0.0216787,2016-08-13,300.671,208.551,275.082
7,3350-09D,0.330728,0.451592,0.502665,2017-10-29,183.238,36.1546,78.8827
8,3410-02D,0.319462,0.483178,0.359859,2013-09-12,262.372,63.9531,100.849
9,4420-04D,0.113083,0.716985,0.250237,2017-07-08,281.439,189.347,229.799


#### Normalize pcp_sum and pcp_max and pcp_max3

In [353]:
min_pcp_sum = minimum(comb.PCP_SUM);
max_pcp_sum = maximum(comb.PCP_SUM);

min_pcp_max = minimum(comb.PCP_MAX);
max_pcp_max = maximum(comb.PCP_MAX);

min_pcp_max3 = minimum(comb.PCP_MAX3);
max_pcp_max3 = maximum(comb.PCP_MAX3);

In [354]:
comb.PCP_SUM = normalize.(comb.PCP_SUM, min_pcp_sum, max_pcp_sum);
comb.PCP_MAX = normalize.(comb.PCP_MAX, min_pcp_max, max_pcp_max);
comb.PCP_MAX3 = normalize.(comb.PCP_MAX3, min_pcp_max3, max_pcp_max3);

In [355]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4230-09D,0.728544,0.88331,0.0,2014-05-27,0.162274,0.0572732,0.0572732
2,3400-01D,0.346518,0.521076,0.578524,2013-05-29,0.155292,0.11966,0.109555
3,4430-04D,0.0424383,0.570669,0.758362,2017-08-04,0.322848,0.441542,0.299637
4,4290-01D,0.447648,0.831152,0.115849,2015-07-01,0.332054,0.142091,0.17112
5,3280-02D,0.519128,0.661874,0.754112,2018-09-21,0.353104,0.26118,0.197466
6,3310-01D,0.51946,0.602926,0.367296,2017-10-04,0.070545,0.0586286,0.0629185
7,4370-03D,0.0842328,0.458181,0.584737,2013-10-17,0.136082,0.0957393,0.131265
8,3310-01D,0.51946,0.602926,0.367296,2016-06-29,0.0200195,0.0214495,0.0200195
9,4400-02D,0.118583,0.711983,0.255796,2016-10-16,0.145603,0.0762682,0.0704903


### Split dates into months and days

In [363]:
comb.MONTH = month.(comb.DATE);
comb.DAY = day.(comb.DATE);
first(shuffleDf(comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Int64,Int64
1,2014-07-10,7,10
2,2017-10-01,10,1
3,2015-05-15,5,15
4,2014-09-22,9,22
5,2015-10-15,10,15


#### Normalize the months and days

In [364]:
comb.MONTH = normalize.(comb.MONTH, 5, 10);
comb.DAY = normalize.(comb.DAY, 1, 31);
first(shuffleDf(comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Float64,Float64
1,2014-10-06,1.0,0.166667
2,2013-06-20,0.2,0.633333
3,2013-07-31,0.4,1.0
4,2018-09-03,0.8,0.0666667
5,2015-09-21,0.8,0.666667


# Validate model

### Split train and validation sets

In [365]:
r_idx = shuffle(1:size(comb, 1));
train_ceil = floor(Int, size(r_idx, 1) * 0.8);
train_set = comb[r_idx[1:train_ceil], :];
val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];

### Train model on train set

#### Build the features and labels

In [375]:
names_ft = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :PCP_MAX, :PCP_MAX3];
train_features = convert(Matrix{Float64},train_set[:, names_ft]);
train_labels = train_set[:, :SURVERSE];

#### Build the model N features to use is log_2(N + 1)

In [381]:
val_model = build_forest(train_labels, train_features, 4, 100, 0.7, 40)

Ensemble of Decision Trees
Trees:      100
Avg Leaves: 2415.73
Avg Depth:  31.58

### Validate model on validation set

#### Single validation

In [382]:
val_features = convert(Matrix{Float64},val_set[:, names_ft]);
val_labels = val_set[!, :SURVERSE];
val_pred = apply_forest(val_model, val_features);

r = roc(val_labels, val_pred);
f1score(r)

0.619348054679285

#### Batch validation for F1Score

In [410]:
niter = 10;
batch_score = 0;

for i=1:niter
    # Split train and val sets
    r_idx = shuffle(1:size(comb, 1));
    train_ceil = floor(Int, size(r_idx, 1) * 0.8);
    train_set = comb[r_idx[1:train_ceil], :];
    val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];
    
    # Build features and labels
    train_features = convert(Matrix{Float64},train_set[:, names_ft]);
    train_labels = train_set[:, :SURVERSE];
    
    # Build model
    val_model = build_forest(train_labels, train_features, 4, 100, 0.7, 40);
    
    # Validate model
    val_features = convert(Matrix{Float64},val_set[:, names_ft]);
    val_labels = val_set[!, :SURVERSE];
    val_pred = apply_forest(val_model, val_features);

    r = roc(val_labels, val_pred);
    batch_score += f1score(r);
end

batch_score = batch_score / niter

0.6169089771652957

# Submission model creation

### Separate features and labels

In [383]:
full_train_features = convert(Matrix{Float64},comb[:, names_ft]);
# train_features = hcat(dates, train_features);

In [384]:
full_train_labels = comb[:, :SURVERSE];

### Build Model

#### Test with tree first

In [385]:
model_tree = build_tree(full_train_labels, full_train_features)

Decision Tree
Leaves: 4072
Depth:  35

In [386]:
model_tree = prune_tree(model_tree, 0.90)

Decision Tree
Leaves: 3930
Depth:  35

In [387]:
model = build_forest(full_train_labels, full_train_features, 4, 100, 0.7, 40)

Ensemble of Decision Trees
Trees:      100
Avg Leaves: 2921.1
Avg Depth:  31.62

# Prediction

## Get the test data

In [389]:
test = CSV.read("data/test.csv");
rename!(test, :NO_OUVRAGE => :ID_OUVRAGE);
first(test, 10)

,ID_OUVRAGE,DATE
,String,Date
1,3260-01D,2019-05-02
2,3260-01D,2019-05-09
3,3260-01D,2019-05-10
4,3260-01D,2019-05-15
5,3260-01D,2019-05-20
6,3260-01D,2019-05-23
7,3260-01D,2019-05-24
8,3260-01D,2019-05-26
9,3260-01D,2019-05-30


In [390]:
to_merge = unique(comb[!, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]], :ID_OUVRAGE);
test_comb = join(test, to_merge, on= [:ID_OUVRAGE]);
nrow(test_comb)

283

In [391]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,DATE,TP_LAT,TP_LNG,TP_Z
,String,Date,Float64,Float64,Float64
1,4380-01D,2019-06-30,0.160421,0.728206,0.359859
2,4350-01D,2019-07-08,0.237379,0.74432,0.359859
3,4240-01D,2019-05-23,0.607546,0.868856,0.116503
4,4350-01D,2019-08-06,0.237379,0.74432,0.359859
5,4240-01D,2019-07-14,0.607546,0.868856,0.116503
6,4350-01D,2019-09-05,0.237379,0.74432,0.359859
7,3260-01D,2019-08-28,0.609996,0.697487,0.386587
8,3350-07D,2019-09-02,0.353054,0.490395,0.405552
9,4240-01D,2019-05-22,0.607546,0.868856,0.116503


### Add PCP_SUM and PCP_MAX

#### Initialize default pcp

In [399]:
test_comb.PCP_SUM = zeros(size(test_comb, 1));
test_comb.PCP_MAX = zeros(size(test_comb, 1));
test_comb.PCP_MAX3 = zeros(size(test_comb, 1));
permutecols!(test_comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX, :PCP_MAX3]);

In [400]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4380-01D,0.160421,0.728206,0.359859,2019-08-17,0.0,0.0,0.0
2,4350-01D,0.237379,0.74432,0.359859,2019-06-14,0.0,0.0,0.0
3,4240-01D,0.607546,0.868856,0.116503,2019-08-13,0.0,0.0,0.0
4,4350-01D,0.237379,0.74432,0.359859,2019-09-10,0.0,0.0,0.0
5,4350-01D,0.237379,0.74432,0.359859,2019-05-30,0.0,0.0,0.0
6,3260-01D,0.609996,0.697487,0.386587,2019-08-01,0.0,0.0,0.0
7,3350-07D,0.353054,0.490395,0.405552,2019-05-11,0.0,0.0,0.0
8,3350-07D,0.353054,0.490395,0.405552,2019-05-13,0.0,0.0,0.0
9,4380-01D,0.160421,0.728206,0.359859,2019-05-28,0.0,0.0,0.0


#### Populate pcp

In [401]:
for i=1:size(test_comb, 1)
    id_ouvrage = test_comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(test_comb[i, :TP_LAT], test_comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([test_comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    test_comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist); 
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([test_comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    test_comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
    # Augment comb with a weighted p_max3, based on the distance to the station
    p_max3 = pcp_max3h[∈([test_comb[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
    test_comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
end

In [402]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4380-01D,0.160421,0.728206,0.359859,2019-06-24,0.0,0.0,0.0
2,3260-01D,0.609996,0.697487,0.386587,2019-08-09,55.153,21.8047,36.5549
3,3350-07D,0.353054,0.490395,0.405552,2019-07-23,0.0,0.0,0.0
4,4240-01D,0.607546,0.868856,0.116503,2019-06-13,5.19718,5.19718,5.19718
5,4350-01D,0.237379,0.74432,0.359859,2019-06-19,0.0,0.0,0.0
6,3350-07D,0.353054,0.490395,0.405552,2019-06-14,45.6263,14.9465,40.9063
7,3260-01D,0.609996,0.697487,0.386587,2019-07-31,0.0,0.0,0.0
8,3350-07D,0.353054,0.490395,0.405552,2019-06-30,24.3865,14.9465,24.3865
9,3350-07D,0.353054,0.490395,0.405552,2019-07-29,0.0,0.0,0.0


#### Normalize pcp based on comb (not test_comb)

In [403]:
test_comb.PCP_SUM = normalize.(test_comb.PCP_SUM, min_pcp_sum, max_pcp_sum);
test_comb.PCP_MAX = normalize.(test_comb.PCP_MAX, min_pcp_max, max_pcp_max);
test_comb.PCP_MAX3 = normalize.(test_comb.PCP_MAX3, min_pcp_max3, max_pcp_max3);

In [404]:
first(test_comb, 20)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,3260-01D,0.609996,0.697487,0.386587,2019-05-02,0.0222322,0.0153915,0.0188119
2,3260-01D,0.609996,0.697487,0.386587,2019-05-09,0.0761026,0.0577183,0.0761026
3,3260-01D,0.609996,0.697487,0.386587,2019-05-10,0.329208,0.0846535,0.132538
4,3260-01D,0.609996,0.697487,0.386587,2019-05-15,0.00171017,0.00256526,0.00171017
5,3260-01D,0.609996,0.697487,0.386587,2019-05-20,0.039334,0.0513052,0.0376238
6,3260-01D,0.609996,0.697487,0.386587,2019-05-23,0.14964,0.0808056,0.122277
7,3260-01D,0.609996,0.697487,0.386587,2019-05-24,0.0111161,0.0115437,0.00940595
8,3260-01D,0.609996,0.697487,0.386587,2019-05-26,0.00256526,0.00384789,0.00256526
9,3260-01D,0.609996,0.697487,0.386587,2019-05-30,0.0059856,0.0089784,0.0059856


#### Split dates into month and day

In [405]:
test_comb.MONTH = month.(test_comb.DATE);
test_comb.DAY = day.(test_comb.DATE);

first(shuffleDf(test_comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Int64,Int64
1,2019-05-24,5,24
2,2019-07-11,7,11
3,2019-09-13,9,13
4,2019-05-06,5,6
5,2019-06-28,6,28


#### Normalize months and days

In [406]:
test_comb.MONTH = normalize.(test_comb.MONTH, 5, 10);
test_comb.DAY = normalize.(test_comb.DAY, 1, 31);
first(shuffleDf(test_comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Float64,Float64
1,2019-08-27,0.6,0.866667
2,2019-07-07,0.4,0.2
3,2019-07-05,0.4,0.133333
4,2019-09-28,0.8,0.9
5,2019-07-31,0.4,1.0


### Create Test features

In [407]:
test_features = convert(Matrix{Float64}, test_comb[:, names_ft]);
# test_features = hcat(test_dates, test_features);

test_features

283×8 Array{Float64,2}:
 0.609996  0.697487  0.386587  0.0  …  0.0222322   0.0153915   0.0188119 
 0.609996  0.697487  0.386587  0.0     0.0761026   0.0577183   0.0761026 
 0.609996  0.697487  0.386587  0.0     0.329208    0.0846535   0.132538  
 0.609996  0.697487  0.386587  0.0     0.00171017  0.00256526  0.00171017
 0.609996  0.697487  0.386587  0.0     0.039334    0.0513052   0.0376238 
 0.609996  0.697487  0.386587  0.0  …  0.14964     0.0808056   0.122277  
 0.609996  0.697487  0.386587  0.0     0.0111161   0.0115437   0.00940595
 0.609996  0.697487  0.386587  0.0     0.00256526  0.00384789  0.00256526
 0.609996  0.697487  0.386587  0.0     0.0059856   0.0089784   0.0059856 
 0.353054  0.490395  0.405552  0.0     0.0492974   0.0330397   0.0430041 
 0.353054  0.490395  0.405552  0.0  …  0.0136354   0.0125866   0.0115377 
 0.353054  0.490395  0.405552  0.0     0.0         0.0         0.0       
 0.353054  0.490395  0.405552  0.0     0.0902037   0.0676528   0.0902037 
 ⋮            

## Predict

In [408]:
test_labels = apply_forest(model, test_features)

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 ⋮
 0
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0

## Generate submission

In [409]:
ID = test_comb[:,:ID_OUVRAGE].*"_".*string.(test_comb[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=test_labels)
CSV.write("submissions/mc-submission-7.csv",sampleSubmission)

"submissions/mc-submission-7.csv"